In [ ]:
import pathlib

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.ticker import PercentFormatter

In [ ]:
root = pathlib.Path().resolve().parent
output_dir = root / "out"
metrics_file = output_dir / "metrics.csv"

df = pd.read_csv(metrics_file)
df["date"] = pd.to_datetime(df["date"])

# Convert percentage strings (e.g. "2.58%") into floats so plotting works
percent_cols = [col for col in df.columns if col.endswith("_percent")]
for col in percent_cols:
    df[col] = pd.to_numeric(df[col].str.rstrip("%"), errors="coerce")

In [ ]:
# lets create a time series plot of the metrics
plt.figure(figsize=(12, 6))

series_to_plot: list[str] = [
    "weighted_mean_percent",
    "weighted_median_percent",
    "trimmed_mean_percent",
    "weighted_kde_mode_percent",
]
for series in series_to_plot:
    plt.plot(df["date"], df[series], label=series)

plt.legend()
plt.title("Time Series of Metrics")
plt.xlabel("Date")
plt.ylabel("Percentage")

ax = plt.gca()
ax.yaxis.set_major_formatter(PercentFormatter(xmax=100, decimals=1))
ymin = df[series_to_plot].min().min()
ymax = df[series_to_plot].max().max()
ax.set_ylim(ymin - 0.5, ymax + 0.5)

plt.grid(True)
plt.show()

In [ ]:
root = pathlib.Path().resolve().parent
output_dir = root / "out"
panel_file = output_dir / "panel.csv"

df = pd.read_csv(panel_file)
df["date"] = pd.to_datetime(df["date"])

# Convert percentage strings (e.g. "2.58%") into floats so plotting works
percent_cols = [col for col in df.columns if col.endswith("_percent")]
for col in percent_cols:
    df[col] = pd.to_numeric(df[col].str.rstrip("%"), errors="coerce")

In [ ]:
sns.set_theme(style="whitegrid", font_scale=1.1)

# --- Create a FacetGrid with one panel per display-level group ---
g = sns.FacetGrid(df, col="group_label", col_wrap=3, height=3.5, sharey=True, aspect=1.4)


# --- Plot the lines for each inflation measure ---
def plot_group(data, **kwargs):
    plt.plot(data["date"], data["mode"], label="KDE Mode", color="darkorange", linewidth=2.2)
    plt.plot(
        data["date"],
        data["mean"],
        label="Weighted Mean",
        color="steelblue",
        linestyle="--",
        linewidth=1.8,
    )
    plt.plot(
        data["date"],
        data["median"],
        label="Weighted Median",
        color="green",
        linestyle=":",
        linewidth=1.8,
    )
    plt.plot(
        data["date"],
        data["trimmed_mean"],
        label="Trimmed Mean",
        color="red",
        linestyle="-.",
        linewidth=1.8,
    )

    # Optional: show volatility as shaded std band
    if "std" in data:
        upper = data["mean"] + data["std"]
        lower = data["mean"] - data["std"]
        plt.fill_between(data["date"], lower, upper, color="steelblue", alpha=0.15)

    plt.title(f"Group {int(data['group_label'].iloc[0])}")
    plt.xlabel("")
    plt.ylabel("Percent")
    plt.tight_layout()


g.map_dataframe(plot_group)

for ax in g.axes.flat:
    # Major ticks every 6 months
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
    # Label format as 'YYYY-MM'
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
    # Minor ticks every 3 months for visual rhythm
    ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=3))
    # Tilt labels slightly to avoid collisions
    for label in ax.get_xticklabels(which="major"):
        label.set_rotation(30)
        label.set_horizontalalignment("right")

# Collect handles and labels from one axis
handles, labels = g.axes[0].get_legend_handles_labels()

# Remove legends from each subplot
for ax in g.axes.flat:
    ax.legend_.remove() if ax.get_legend() else None

# Add a single legend beneath all subplots
g.fig.legend(
    handles,
    labels,
    loc="lower center",
    ncol=4,
    frameon=False,
    fontsize=10,
    bbox_to_anchor=(0.5, -0.02),
)

# Adjust bottom margin so legend isn't clipped
plt.subplots_adjust(bottom=0.10, top=0.90)

# Set global labels
g.set_titles("Display Level {col_name}")
g.set_axis_labels("Date", "Inflation (%)")

plt.subplots_adjust(top=0.92)
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(PercentFormatter(xmax=100, decimals=1))
g.figure.suptitle("KDE Mode vs Traditional Inflation Metrics by BLS Display Level", fontsize=16)
plt.show()